In [ ]:
import scipy.io
import numpy as np
import torch
from NavStokesPINN import NavStokesPINN

In [ ]:
np.random.seed(42)
torch.set_default_dtype(torch.float)
torch.manual_seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
N_train = 5000

layers = [3, 20, 20, 20, 20, 20, 20, 20, 20, 2]
data = scipy.io.loadmat('Data/cylinder_nektar_wake.mat')

In [ ]:
U_star = data['U_star']     # N x 2 x T
P_star = data['p_star']     # N x T
t_star = data['t']          # T x 1
X_star = data['X_star']     # N x 2

In [ ]:
N = X_star.shape[0]
T = t_star.shape[0]

In [ ]:
# rearrange data

# inputs
XX = np.tile(X_star[:,0:1], (1,T))      # N x T
YY = np.tile(X_star[:,1:2], (1,T))      # N x T
TT = np.tile(t_star, (1,N)).T           # N x T

# outputs
UU = U_star[:,0,:]                      # N x T
VV = U_star[:,1,:]                      # N x T
PP = P_star                             # N x T

In [ ]:
# flatten data

x = XX.flatten()[:,None]                # NT x 1
y = YY.flatten()[:,None]                # NT x 1
t = TT.flatten()[:,None]                # NT x 1

u = UU.flatten()[:,None]
v = VV.flatten()[:,None]
p = PP.flatten()[:,None]

In [ ]:
# training data selection

idx = np.random.choice(N*T, N_train, replace=False)     # randomly sampling training data from our full dataset
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

In [ ]:
model = NavStokesPINN(x_train, y_train, t_train, u_train, v_train, layers, device)
model.to(device)
print(model)
model.train(100)
print(model.lambda_1)
print(model.lambda_2)